# Differential Expression Gene Analysis for RapidDx Pilot

### Libraries and packages Installation

1) Install packages:

In [ ]:
install.packages("gplots")
install.packages("DT")
install.packages("calibrate")
install.packages("statmod")

2) Install Bioconductor libraries:

In [ ]:
library(BiocInstaller)
biocLite()
biocLite("impute") # requirements for WGCNA package
biocLite("GO.db")
biocLite("preprocessCore")
biocLite("limma")
biocLite("lumi")

In [ ]:
install.packages("WGCNA")

## GENE EXPRESSION ANALYSIS

#### Import libraries

In [ ]:
library(RColorBrewer) # Colour palette
library(xtable)       # HTML tables for Rmarkdown, http://kbroman.org/knitr_knutshell/pages/figs_tables.html
library(readr)
library(readxl)   
library(plyr) 
library(calibrate)
library(WGCNA)        # Gene modules
library(gplots)       # Pretty heatmaps
library(DT)
library(limma) 
library(biomaRt)

In [ ]:
table_html_attributes <- 'border="3" align="center" style="border-collapse: collapse; text-align: right; width: 75%; background-color: #f7f7f7; border-color: #cccccc; "' ## HTML/CSS for xtable formatting

#### Get study files:
1. **Study:** GSE63878
2. **Platform:** GPL6244 = [HuGene-1_0-st] Affymetrix Human Gene 1.0 ST Array [transcript (gene) version]

In [ ]:
source('brain_api_functions.R')
GSE63878 <- download_data('GSE63878')
dataA=GSE63878
head(dataA)

In [ ]:
row.names(dataA)=unlist(dataA[,2])
names(dataA)[names(dataA)=="Gene Symbol"]="Symbol"
dim(dataA)

In [ ]:
class(dataA$Symbol)
exprs.max <- apply(dataA, 1, max)


#### Remove outliers:

In [ ]:
beta.normA=dataA[,-c(1,2,99, 100, 101, 102,103)]
beta.normA= sapply(beta.normA, as.numeric)
row.names(beta.normA)=unlist(dataA[,2]) # then fix row names
dim(beta.normA)

#### Load phenotypes:

In [ ]:
#pheno_63878 <- read_delim("pheno_63878_2.txt", "\t", escape_double = FALSE, trim_ws = TRUE)
pheno_63878 <- get_phenodata('bhc-RAPID-DxPilot', 'GSE63878')
phenoA=pheno_63878
row.names(phenoA)=unlist(phenoA[,2])
phenoA <- data.frame(phenoA)[colnames(beta.normA),]
dim(phenoA)

### PRE-DEPLOYMENT ANALYSIS

In [ ]:
phenoXA = phenoA[phenoA$Time==1 ,]#Selecting Time 1=pre-deployment
# Adjusting the expression matrix to the selected subjects
beta.normXA<- beta.normA[,rownames(phenoXA)]
# Count subjects used in the analysis
count(phenoXA$PTSD==1)
count(phenoXA$PTSD==2)

#### Step1: Create Design Matrix: 
- Age (if applicable)
- Gender (if applicable)
- PCs from Pop strat (if applicable) 
- Cell types (if applicable)
- PTSD Current


In [ ]:
design<-model.matrix(~PTSD, data=phenoXA)
colnames(design)[2] <- "PTSD"

#### Step 2: Run Model

In [ ]:
fit<-lmFit(beta.normXA, design) # Runs linear models
fit.coef<-fit$coef # Extracts the beta coefficients

Calculate the number of subjects with complete beta values

In [ ]:
N.subjects<-apply(beta.normXA, 1, function(x) sum(!is.na(x)))
fit.coef<-cbind(fit.coef, N.subjects) # Adds N subjects to the coefficients for use in weighting
contrast.matrix<-makeContrasts(PTSD, levels=design)
contrast.matrix # Contrast matrix extracts only the coefficient we are interested in
rownames(contrast.matrix)[1]<-"Intercept" # Have to rename the contrast matrix
contrast.matrix

#### Step 3: Get top expressed genes and plots

In [ ]:
fit2<-contrasts.fit(fit, contrast.matrix)
fit2.ebayes<-eBayes(fit2) # Run empirical bayes
results1=topTable(fit2.ebayes,number=nrow(fit2.ebayes),coef=1, adjust="BH")
save(fit.coef, fit2.ebayes, results1, file="Array_1.RData") # These will be used in meta-analysis
topTable(fit2.ebayes,n=10,coef=1, adjust="BH")

Make a **basic volcano plot**

In [ ]:
res <- results1
names <- rownames(res)
rownames(res) <- NULL
res <- cbind(names,res)

par(mfrow=c(1,1))
with(res, plot(logFC, -log10(P.Value), pch=20, main="PTSD_Time1: Volcano plot", xlim=c(-1.0,1.0)))

# Add colored points: red if adj.P.Val<0.05, orange of log2FC>1, green if both)
with(subset(res, P.Value>.05 ), points(logFC, -log10(P.Value), pch=20, col="grey"))
with(subset(res, P.Value<.05 & logFC<0), points(logFC, -log10(P.Value), pch=20, col="light green"))
with(subset(res, P.Value<.05 & logFC>0), points(logFC, -log10(P.Value), pch=20, col="pink"))
with(subset(res, adj.P.Val<.05 & logFC<0), points(logFC, -log10(P.Value), pch=20, col="green"))
with(subset(res, adj.P.Val<.05 & logFC>0), points(logFC, -log10(P.Value), pch=20, col="red"))

# Label points with the textxy function from the calibrate plot
with(subset(res, P.Value <0.001), textxy(logFC, -log10(P.Value), labs=names, cex=1))
with(subset(res, P.Value <0.01 & abs(logFC)>2.0), textxy(logFC, -log10(P.Value), labs=names, cex=1))

Plot **heatmap**

In [ ]:
results1_2=results1[results1$P.Val< 0.05 & abs(results1$logFC)>0.3,] #p<0.05
tphenoXA=t(phenoXA)
heat_T=heatmap.2(beta.normXA[rownames(results1_2),],                
                 trace="none", density="none", col=bluered(50), cexRow=1, cexCol=1, margins = c(5.0,5.0),
                 ColSideColors=tphenoXA["PTSD",], scale="row")

### POST-DEPLOYMENT ANALYSIS

In [ ]:
phenoXA = phenoA[phenoA$Time==2,] # Selecting Time 2=post-deployment
# Adjusting the expression matrix to the selected subjects
beta.normXA<- beta.normA[,rownames(phenoXA)]
# Count subjects used in the analysis
count(phenoXA$PTSD==1)
count(phenoXA$PTSD==2)

#### Step1: Create Design Matrix

In [ ]:
design<-model.matrix(~PTSD, data=phenoXA)
colnames(design)[2] <- "PTSD"

#### Step 2: Run Model

In [ ]:
fit<-lmFit(beta.normXA, design) # Runs linear models
fit.coef<-fit$coef # Extracts the beta coefficients

Calculate the number of subjects with complete beta values

In [ ]:
N.subjects<-apply(beta.normXA, 1, function(x) sum(!is.na(x)))

fit.coef<-cbind(fit.coef, N.subjects) # Adds N subjects to the coefficients for use in weighting
contrast.matrix<-makeContrasts(PTSD, levels=design) # HERE HERE
contrast.matrix # Contrast matrix extracts only the coefficient we are interested in
rownames(contrast.matrix)[1]<-"(Intercept)" # Have to rename the contrast matrix
contrast.matrix

#### Step 3: Get top expressed genes and plots

In [ ]:
fit2<-contrasts.fit(fit, contrast.matrix)
fit2.ebayes<-eBayes(fit2) # Run empirical bayes
results2=topTable(fit2.ebayes,number=nrow(fit2.ebayes),coef=1, adjust="BH")
save(fit.coef, fit2.ebayes, results2, file="Array_2.RData") #these will be used in meta-analysis
topTable(fit2.ebayes,n=10,coef=1, adjust="BH")

Make a **basic volcano plot**

In [ ]:
res <- results2
names <- rownames(res)
rownames(res) <- NULL
res <- cbind(names,res)

par(mfrow=c(1,1))
with(res, plot(logFC, -log10(P.Value), pch=20, main="PTSD_Time2: Volcano plot", xlim=c(-1.0,1.0)))

# Add colored points: red if adj.P.Val<0.05, orange of log2FC>1, green if both)
with(subset(res, P.Value>.05 ), points(logFC, -log10(P.Value), pch=20, col="grey"))
with(subset(res, P.Value<.05 & logFC<0), points(logFC, -log10(P.Value), pch=20, col="light green"))
with(subset(res, P.Value<.05 & logFC>0), points(logFC, -log10(P.Value), pch=20, col="pink"))
with(subset(res, adj.P.Val<.05 & logFC<0), points(logFC, -log10(P.Value), pch=20, col="green"))
with(subset(res, adj.P.Val<.05 & logFC>0), points(logFC, -log10(P.Value), pch=20, col="red"))

# Label points with the textxy function from the calibrate plot
with(subset(res, P.Value <0.001), textxy(logFC, -log10(P.Value), labs=names, cex=1))
with(subset(res, P.Value <0.01 & abs(logFC)>0.3), textxy(logFC, -log10(P.Value), labs=names, cex=1))

Plot **heatmap**. Label points with the textxy function from the calibrate plot

In [ ]:
results2_2=results2[results2$P.Val< 0.05 & abs(results2$logFC)>0.3,] #p<0.05
tphenoXA=t(phenoXA)
heat_T=heatmap.2(beta.normXA[rownames(results2_2),],
                 trace="none", density="none", col=bluered(50), cexRow=1, cexCol=1, margins = c(5.0,5.0),
                 ColSideColors=tphenoXA["PTSD",], scale="row")

### LONGITUDINAL ANALYSIS

In [ ]:
phenoXA=phenoA
beta.normXA=beta.normA

#### Step1: Create Design Matrix

In [ ]:
Treat <- factor(paste(phenoXA$PTSD,phenoXA$Time,sep="."))
design <- model.matrix(~0+Treat, data=phenoXA)

#### Step2: Run model

In [ ]:
corfit <- duplicateCorrelation(beta.normXA,design,block=phenoXA$FactorValue..individual.) 
corfit$consensus
fit <- lmFit(beta.normXA,design,block=phenoXA$FactorValue..individual.,correlation=corfit$consensus)

#### Step 3: Get top expressed genes and plots

In [ ]:
cm <-makeContrasts(
  PTSDvsControl = (Treat2.2-Treat1.2)-(Treat2.1 - Treat1.1),
  PTSD = Treat2.2-Treat2.1,
  Control = Treat1.2-Treat1.1,
  levels=design)

In [ ]:
fit2 <- contrasts.fit(fit, cm)
fit2 <- eBayes(fit2)

topTable(fit2, coef="PTSDvsControl")
topTable(fit2, coef="PTSD")
topTable(fit2, coef="Control")

longPTSDvsControl<- topTable(fit2, coef="PTSDvsControl", n=80000)
longPTSD<- topTable(fit2, coef="PTSD", n=80000)
longControl<- topTable(fit2, coef="Control", n=80000)

In [ ]:
res <- longPTSDvsControl
names <- rownames(res)
rownames(res) <- NULL
res <- cbind(names,res)

** Volcano plot**

In [ ]:
par(mfrow=c(1,1))
# Add colored points: red if adj.P.Val<0.05, orange of log2FC>1, green if both)
with(res, plot(logFC, -log10(P.Value), pch=20, main="PTSD_Longtitudinal: Volcano plot", xlim=c(-1.0,1.0)))
with(subset(res, P.Value>.05 ), points(logFC, -log10(P.Value), pch=20, col="grey"))
with(subset(res, P.Value<.05 & logFC<0), points(logFC, -log10(P.Value), pch=20, col="light green"))
with(subset(res, P.Value<.05 & logFC>0), points(logFC, -log10(P.Value), pch=20, col="pink"))
with(subset(res, adj.P.Val<.05 & logFC<0), points(logFC, -log10(P.Value), pch=20, col="green"))
with(subset(res, adj.P.Val<.05 & logFC>0), points(logFC, -log10(P.Value), pch=20, col="red"))
# Label points with the textxy function from the calibrate plot
with(subset(res, P.Value <0.01), textxy(logFC, -log10(P.Value), labs=names, cex=1))

** Heatmap **

In [ ]:
results3_2=longPTSDvsControl[longPTSDvsControl$P.Val< 0.025,] #p<0.05
tphenoXA=t(phenoXA)

In [ ]:
heat_T=heatmap.2(beta.normXA[rownames(results3_2),],
                 trace="none", density="none", col=bluered(50), cexRow=1, cexCol=1, margins = c(5.0,5.0),
                 ColSideColors=tphenoXA["PTSD",], scale="row")